In [166]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate

# from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from embedchain import App
from langchain_community.embeddings import OllamaEmbeddings
# Local Vector Database
from langchain_chroma import Chroma
from langchain.tools import tool

from langchain_community.llms import Ollama

from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from langchain_community.tools import DuckDuckGoSearchRun

from duckduckgo_search import DDGS


from crewai_tools import FirecrawlSearchTool
from crewai_tools import FirecrawlCrawlWebsiteTool



In [159]:
# Load ENV
from dotenv import load_dotenv
import os
load_dotenv()
# ANTHROPIC_API_KEY
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
# FIRECRAWL_KEY
FIRECRAWL_KEY = os.getenv("FIRECRAWL_KEY")
# SEARCH_KEY
SEARCH_KEY = os.getenv("SEARCH_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# declare Model
claude_sonnet_model = "claude-3-sonnet-20240229" # effective one
claude_haiku_model = "claude-3-haiku-20240307" # small one

sonnet_model = ChatAnthropic(model=claude_sonnet_model, api_key=ANTHROPIC_API_KEY)
haiku_model = ChatAnthropic(model=claude_haiku_model, api_key=ANTHROPIC_API_KEY)
ollama_llm = Ollama(model="llama3.1:8b")
openai_llm = ChatOpenAI(model="gpt-4o-mini")

llama_embedding_function = OllamaEmbeddings(model="llama3.1:8b")  # using local model in my laptop
openai_embedding_function = OpenAIEmbeddings()


In [109]:
# Define Vector DB as a local storage and embedding function
CHROMA_PATH="data/chroma"

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=openai_embedding_function)

class GetNews:
    @tool("Get News Tool")
    def news(query: str) -> str:
        """Search Chroma DB for relevant news information based on a query."""
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=openai_embedding_function)
        retriever = vectorstore.similarity_search(query)
        return retriever

In [23]:
# # Set Up Crawler
# from firecrawl import FirecrawlApp

# app = FirecrawlApp(api_key=FIRECRAWL_KEY)
# # Scrape a website:
# links = [
#     "https://www.nhc.noaa.gov/cyclones/"
# ]
# scrape_result = app.scrape_url(links[0], params={'formats': ['markdown', 'html']})
# print(scrape_result)
# embedding data into database
# db.embed(scrape_result['text'], scrape_result['url'])


{'content': '[![Skip Navigation Links](https://www.nhc.noaa.gov/css/images/skipgraphic.gif)](#contents)\n\n[Home](/index.shtml)\n\n[Mobile Site](/mobile)\n\n[Text Version](?text)\n\n[RSS](/aboutrss.shtml)\n\n \n\n[Local Forecast](https://www.weather.gov)\n\n[![NOAA](https://www.nhc.noaa.gov/css/images/noaa-emblem-70x75.png)](https://www.noaa.gov)\n [![NOAA](https://www.nhc.noaa.gov/css/images/header-nws.png)](https://www.weather.gov)\n [![United States Department of Commerce](https://www.nhc.noaa.gov/css/images/header_doc.png)](https://www.commerce.gov)\n\n[NATIONAL HURRICANE CENTER and  \\\nCENTRAL PACIFIC HURRICANE CENTER](/index.php)\n\n[National Oceanic and Atmospheric Administration](https://www.noaa.gov)\n\n*   [Analyses & Forecasts](/cyclones)\n    \n    *   [Tropical Cyclone Products](/cyclones)\n        \n    *   [Tropical Weather Outlooks](/gtwo.php)\n        \n    *   [Marine Products](/marine)\n        \n    *   [Audio/Podcasts](/audio)\n        \n    *   [RSS Feeds](/about

In [110]:
# Set Up CrewAI <-- Crew, Agent, Task
#  declare crew library
from crewai import Crew, Agent, Task, Process


In [141]:
#  declare tools right here


web_cralw_tool = FirecrawlCrawlWebsiteTool(url='https://www.nhc.noaa.gov/cyclones/', api_key=FIRECRAWL_KEY)
fire_crawl_search_tool = FirecrawlSearchTool(api_key=FIRECRAWL_KEY)
duck_duck_go_search_tool = DuckDuckGoSearchRun()


class search:
    @tool("GetCompany")
    def company(query: str) -> str:
        """Search Chroma DB for relevant news information based on a query."""
        result = duck_duck_go_search_tool.search(query)
        return result

In [161]:
# 1st Agent [สมศรี]
information_agent = Agent(
    role='Information And Searching Analyst',
    goal='Analyze the weather website and extract relevant information about the weather and our needs to create tornado alerts for our client companies. from this page =https://www.nhc.noaa.gov/cyclones and look up for some upcoming tornado in 12 hours',
    tools=[web_cralw_tool, duck_duck_go_search_tool],
    backstory='Expert in analyzing weather and exacting information from websites to create tornado alerts for our client companies. result in how likely the tornado will be happened in 5 level 1-5 and list tornado name',
    verbose=True,
    llm=sonnet_model
    )

# 2nd Agent [สมศักดิ์]   
company_contractor_agent = Agent(
    role='Data Analyst',
    goal='To Search company by google and let contract them as fast as possible to let them know about incoming alert tornado. predict how',
    tools=[duck_duck_go_search_tool],
    backstory='Excel and fast in searching company by google and let contract them as fast as possible to let them know about incoming alert tornado.',
    verbose=True,
    llm=sonnet_model
    )


# 3rd Agent [สมพร]
customer_service_agent = Agent(
    role='Customer Service Representative',
    goal='Provide timely and accurate support to customers by addressing their inquiries, resolving issues, and ensuring a positive experience. The agent should efficiently handle common customer service tasks such as answering questions, processing orders, and managing complaints.',
    backstory = "The agent is an experienced customer service representative with extensive knowledge of the company's products and services. They are empathetic, patient, and skilled in problem-solving, dedicated to helping customers find solutions quickly and effectively. The agent is trained to handle a wide range of scenarios, from simple inquiries to complex issues, ensuring customer satisfaction at every interaction.",
    verbose=True,
    llm=sonnet_model
    )


In [165]:
# Define Task

information_task = Task(
    description="Analyze the weather website and extract relevant information about the weather and our needs to create tornado alerts for our client companies.",
    agent=information_agent,
    verbose=True,
    tools=[web_cralw_tool,duck_duck_go_search_tool],
    expected_output='A refined finalized version of tornado name, what when and how coming time as a information exact as text list'
)

company_contractor_task = Task(
    description="Find utility companies from the internet in the area from weather incoming just only once search",
    agent=company_contractor_agent,
    tools=[duck_duck_go_search_tool],
    verbose=True,
    expected_output='list the closest utility companies that help improve and repair utility services that could be impact by location of search from the alert weather incoming'
)

customer_service_task = Task(
    description="Analyze Agent and set them to work",
    agent=customer_service_agent,
    verbose=True,
    expected_output='1.refine what the tornado name coming as a bullet list. 2.what company effect and process what company will handle tornado by area in the US'
)

In [163]:
crew = Crew(
    agents=[information_agent,company_contractor_agent, customer_service_agent],
    tasks=[information_task,company_contractor_task, customer_service_task],
    process=Process.sequential,
    verbose=2
)



2024-08-29 22:44:07,384 - 8346799104 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [167]:
result = crew.kickoff()

 [2024-08-29 23:35:04][DEBUG]: == Working Agent: Information And Searching Analyst
 [2024-08-29 23:35:04][INFO]: == Starting Task: Analyze the weather website and extract relevant information about the weather and our needs to create tornado alerts for our client companies.


> Entering new CrewAgentExecutor chain...
Thought: To gather relevant information about potential tornados from the provided website, I should first use the Firecrawl web crawl tool to retrieve the webpage content.

Action: Firecrawl web crawl tool
Action Input: {"url": "https://www.nhc.noaa.gov/cyclones"}
 

I encountered an error while trying to use the tool. This was the error: Unexpected error during start crawl job: Status code 429. Rate limit exceeded. Consumed (req/min): 6, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 58s, resets at Fri Aug 30 2024 06:36:07 GMT+0000 (Coordinated Universal Time).
 Tool Firecrawl web crawl tool acce

In [150]:
result.raw

'- Tornado Name: Based on the information from the NOAA website, there is currently no active tornado threat that would impact the CenterPoint Energy and Great Lakes Energy service areas. The utility companies are closely monitoring weather conditions and will be ready to respond if a tornado does develop in their regions.\n\nCompany Effect:\n- CenterPoint Energy (Texas): This utility company serves the Houston, Texas area and has over 300,000 customers who recently lost power due to devastating storms. CenterPoint Energy has an extensive emergency response plan in place to prepare for and respond to severe weather events, including tornadoes. They will closely coordinate with local authorities and emergency management agencies to ensure the safety of their customers and employees.\n\n- Great Lakes Energy (Michigan): This utility company serves southwest Michigan, including areas near the Indiana border that could be impacted by a tornado. Great Lakes Energy also has robust emergency p

In [168]:
result_str = result.raw
# split result by \n and loop print
result_list = result_str.split("\n")
for result_line in result_list:
    print(result_line)

Tornado Preparedness and Response:
- Major utility companies in tornado-prone regions have established procedures and protocols to prepare for and respond to tornado events.
- These protocols typically involve monitoring weather alerts, securing and reinforcing infrastructure, mobilizing repair crews, and coordinating with emergency services.
- Utility companies may also issue public advisories and safety tips to customers in affected areas.

Major Utility Companies Operating in Tornado-Prone Regions:
- Oklahoma Gas & Electric (OG&E) - Serves customers in Oklahoma and western Arkansas.
- Evergy (formerly Westar Energy and Kansas City Power & Light) - Serves customers in Kansas and Missouri.
- Florida Power & Light (FPL) - Serves customers in Florida, which is prone to tornadoes during hurricane season.
- Entergy Corporation - Serves customers in Arkansas, Louisiana, Mississippi, and Texas, which are part of the traditional "Tornado Alley" region.
- Duke Energy - Serves customers in Nor